电商18 180412126 王佳琦

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()

In [22]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)  #随机梯度下降法

In [12]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.1")
visdom_accu = Visdom(env="testaccu_0.1")
visdom_test = Visdom(env="testloss_0.1")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.1',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.1']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.1']),
            name='test_accu_0.1',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.1']),
            name='test_loss_0.1',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.364
[1,  4000] loss: 2.362
[1,  6000] loss: 2.356
[1,  8000] loss: 2.362
[1, 10000] loss: 2.365
[1, 12000] loss: 2.358
Accuracy of the network on the 2500 test images: 10 %
[1] loss: 2.381
12499
[2,  2000] loss: 2.359
[2,  4000] loss: 2.357
[2,  6000] loss: 2.357
[2,  8000] loss: 2.361
[2, 10000] loss: 2.360
[2, 12000] loss: 2.359
Accuracy of the network on the 2500 test images: 10 %
[2] loss: 2.368
12499
[3,  2000] loss: 2.358
[3,  4000] loss: 2.358
[3,  6000] loss: 2.359
[3,  8000] loss: 2.359
[3, 10000] loss: 2.358
[3, 12000] loss: 2.361
Accuracy of the network on the 2500 test images: 10 %
[3] loss: 2.343
12499
[4,  2000] loss: 2.356
[4,  4000] loss: 2.361
[4,  6000] loss: 2.359
[4,  8000] loss: 2.359
[4, 10000] loss: 2.356
[4, 12000] loss: 2.359
Accuracy of the network on the 2500 test images: 10 %
[4] loss: 2.341
12499
[5,  2000] loss: 2.360
[5,  4000] loss: 2.361
[5,  6000] loss: 2.361
[5,  8000] loss: 2.358
[5, 10000] loss: 2.364
[5, 12000] loss: 2.360
Accura

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)  #随机梯度下降法

In [18]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.01")
visdom_accu = Visdom(env="testaccu_0.01")
visdom_test = Visdom(env="testloss_0.01")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.01',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.01']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.01']),
            name='test_accu_0.01',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.01']),
            name='test_loss_0.01',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.081
[1,  4000] loss: 1.965
[1,  6000] loss: 1.971
[1,  8000] loss: 1.977
[1, 10000] loss: 1.996
[1, 12000] loss: 2.008
Accuracy of the network on the 2500 test images: 27 %
[1] loss: 1.919
12499
[2,  2000] loss: 1.983
[2,  4000] loss: 2.005
[2,  6000] loss: 1.984
[2,  8000] loss: 2.010
[2, 10000] loss: 2.048
[2, 12000] loss: 2.024
Accuracy of the network on the 2500 test images: 24 %
[2] loss: 1.979
12499
[3,  2000] loss: 2.009
[3,  4000] loss: 2.074
[3,  6000] loss: 2.044
[3,  8000] loss: 2.212
[3, 10000] loss: 2.168
[3, 12000] loss: 2.152
Accuracy of the network on the 2500 test images: 18 %
[3] loss: 2.072
12499
[4,  2000] loss: 2.093
[4,  4000] loss: 2.059
[4,  6000] loss: 2.112
[4,  8000] loss: 2.131
[4, 10000] loss: 2.199
[4, 12000] loss: 2.118
Accuracy of the network on the 2500 test images: 20 %
[4] loss: 2.057
12499
[5,  2000] loss: 2.053
[5,  4000] loss: 2.049
[5,  6000] loss: 2.034
[5,  8000] loss: 2.024
[5, 10000] loss: 1.996
[5, 12000] loss: 2.012
Accura

In [23]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [24]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.001")
visdom_accu = Visdom(env="testaccu_0.001")
visdom_test = Visdom(env="testloss_0.001")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.001',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.001']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.001']),
            name='test_accu_0.001',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.001']),
            name='test_loss_0.001',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.197
[1,  4000] loss: 1.898
[1,  6000] loss: 1.712
[1,  8000] loss: 1.579
[1, 10000] loss: 1.524
[1, 12000] loss: 1.473
Accuracy of the network on the 2500 test images: 44 %
[1] loss: 1.514
12499
[2,  2000] loss: 1.403
[2,  4000] loss: 1.375
[2,  6000] loss: 1.327
[2,  8000] loss: 1.337
[2, 10000] loss: 1.313
[2, 12000] loss: 1.283
Accuracy of the network on the 2500 test images: 54 %
[2] loss: 1.250
12499
[3,  2000] loss: 1.222
[3,  4000] loss: 1.200
[3,  6000] loss: 1.215
[3,  8000] loss: 1.199
[3, 10000] loss: 1.177
[3, 12000] loss: 1.162
Accuracy of the network on the 2500 test images: 58 %
[3] loss: 1.183
12499
[4,  2000] loss: 1.098
[4,  4000] loss: 1.118
[4,  6000] loss: 1.093
[4,  8000] loss: 1.096
[4, 10000] loss: 1.091
[4, 12000] loss: 1.082
Accuracy of the network on the 2500 test images: 60 %
[4] loss: 1.127
12499
[5,  2000] loss: 0.994
[5,  4000] loss: 1.012
[5,  6000] loss: 1.035
[5,  8000] loss: 1.041
[5, 10000] loss: 1.048
[5, 12000] loss: 1.030
Accura

使用relu as 激活函数

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [6]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_relu")
visdom_accu = Visdom(env="testaccu_relu")
visdom_test = Visdom(env="testloss_relu")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_relu',
                    opts=dict(title='accu and loss',  legend=['train_loss_relu']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_relu']),
            name='test_accu_relu',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_relu']),
            name='test_loss_relu',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.223
[1,  4000] loss: 1.877
[1,  6000] loss: 1.669
[1,  8000] loss: 1.563
[1, 10000] loss: 1.508
[1, 12000] loss: 1.472
Accuracy of the network on the 2500 test images: 48 %
[1] loss: 1.398
12499
[2,  2000] loss: 1.382
[2,  4000] loss: 1.355
[2,  6000] loss: 1.349
[2,  8000] loss: 1.322
[2, 10000] loss: 1.304
[2, 12000] loss: 1.296
Accuracy of the network on the 2500 test images: 51 %
[2] loss: 1.339
12499
[3,  2000] loss: 1.233
[3,  4000] loss: 1.200
[3,  6000] loss: 1.214
[3,  8000] loss: 1.219
[3, 10000] loss: 1.171
[3, 12000] loss: 1.177
Accuracy of the network on the 2500 test images: 55 %
[3] loss: 1.298
12499
[4,  2000] loss: 1.123
[4,  4000] loss: 1.127
[4,  6000] loss: 1.127
[4,  8000] loss: 1.120
[4, 10000] loss: 1.129
[4, 12000] loss: 1.105
Accuracy of the network on the 2500 test images: 60 %
[4] loss: 1.120
12499
[5,  2000] loss: 1.021
[5,  4000] loss: 1.040
[5,  6000] loss: 1.049
[5,  8000] loss: 1.075
[5, 10000] loss: 1.064
[5, 12000] loss: 1.049
Accura

使用sigmoid as 激活函数

In [10]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.sigmoid(self.conv1(x)))
        x = self.pool(F.sigmoid(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [12]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_sigmoid")
visdom_accu = Visdom(env="testaccu_sigmoid")
visdom_test = Visdom(env="testloss_sigmoid")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_sigmoid',
                    opts=dict(title='accu and loss',  legend=['train_loss_sigmoid']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_sigmoid']),
            name='test_accu_sigmoid',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_sigmoid']),
            name='test_loss_sigmoid',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,  2000] loss: 2.315
[1,  4000] loss: 2.314
[1,  6000] loss: 2.313
[1,  8000] loss: 2.312
[1, 10000] loss: 2.310
[1, 12000] loss: 2.310
Accuracy of the network on the 2500 test images: 10 %
[1] loss: 2.309
12499
[2,  2000] loss: 2.310
[2,  4000] loss: 2.310
[2,  6000] loss: 2.309
[2,  8000] loss: 2.309
[2, 10000] loss: 2.307
[2, 12000] loss: 2.307
Accuracy of the network on the 2500 test images: 10 %
[2] loss: 2.311
12499
[3,  2000] loss: 2.307
[3,  4000] loss: 2.307
[3,  6000] loss: 2.307
[3,  8000] loss: 2.306
[3, 10000] loss: 2.306
[3, 12000] loss: 2.305
Accuracy of the network on the 2500 test images: 10 %
[3] loss: 2.305
12499
[4,  2000] loss: 2.305
[4,  4000] loss: 2.305
[4,  6000] loss: 2.305
[4,  8000] loss: 2.305
[4, 10000] loss: 2.305
[4, 12000] loss: 2.304
Accuracy of the network on the 2500 test images: 10 %
[4] loss: 2.303
12499
[5,  2000] loss: 2.304
[5,  4000] loss: 2.305
[5,  6000] loss: 2.305
[5,  8000] loss: 2.304
[5, 10000] loss: 2.303
[5, 12000] loss: 2.304
Accura

使用 tanh as 激活函数

In [16]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [18]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_tanh")
visdom_accu = Visdom(env="testaccu_tanh")
visdom_test = Visdom(env="testloss_tanh")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_tanh',
                    opts=dict(title='accu and loss',  legend=['train_loss_tanh']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_tanh']),
            name='test_accu_tanh',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_tanh']),
            name='test_loss_tanh',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[1,  2000] loss: 1.973
[1,  4000] loss: 1.672
[1,  6000] loss: 1.572
[1,  8000] loss: 1.519
[1, 10000] loss: 1.486
[1, 12000] loss: 1.418
Accuracy of the network on the 2500 test images: 50 %
[1] loss: 1.373
12499
[2,  2000] loss: 1.375
[2,  4000] loss: 1.336
[2,  6000] loss: 1.322
[2,  8000] loss: 1.304
[2, 10000] loss: 1.290
[2, 12000] loss: 1.279
Accuracy of the network on the 2500 test images: 53 %
[2] loss: 1.329
12499
[3,  2000] loss: 1.208
[3,  4000] loss: 1.235
[3,  6000] loss: 1.194
[3,  8000] loss: 1.203
[3, 10000] loss: 1.196
[3, 12000] loss: 1.180
Accuracy of the network on the 2500 test images: 57 %
[3] loss: 1.204
12499
[4,  2000] loss: 1.131
[4,  4000] loss: 1.147
[4,  6000] loss: 1.110
[4,  8000] loss: 1.113
[4, 10000] loss: 1.142
[4, 12000] loss: 1.112
Accuracy of the network on the 2500 test images: 59 %
[4] loss: 1.148
12499
[5,  2000] loss: 1.048
[5,  4000] loss: 1.064
[5,  6000] loss: 1.061
[5,  8000] loss: 1.069
[5, 10000] loss: 1.066
[5, 12000] loss: 1.066
Accura

探索relu变种

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [4]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_relu")
visdom_accu = Visdom(env="testaccu_relu")
visdom_test = Visdom(env="testloss_relu")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_relu',
                    opts=dict(title='accu and loss',  legend=['train_loss_relu']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_relu']),
            name='test_accu_relu',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_relu']),
            name='test_loss_relu',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...
D:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 2.282
[1,  4000] loss: 1.886
[1,  6000] loss: 1.691
[1,  8000] loss: 1.590
[1, 10000] loss: 1.526
[1, 12000] loss: 1.461
Accuracy of the network on the 2500 test images: 48 %
[1] loss: 1.412
12499
[2,  2000] loss: 1.406
[2,  4000] loss: 1.364
[2,  6000] loss: 1.332
[2,  8000] loss: 1.325
[2, 10000] loss: 1.299
[2, 12000] loss: 1.272
Accuracy of the network on the 2500 test images: 54 %
[2] loss: 1.299
12499
[3,  2000] loss: 1.210
[3,  4000] loss: 1.212
[3,  6000] loss: 1.162
[3,  8000] loss: 1.194
[3, 10000] loss: 1.178
[3, 12000] loss: 1.166
Accuracy of the network on the 2500 test images: 58 %
[3] loss: 1.163
12499
[4,  2000] loss: 1.105
[4,  4000] loss: 1.092
[4,  6000] loss: 1.117
[4,  8000] loss: 1.101
[4, 10000] loss: 1.094
[4, 12000] loss: 1.094
Accuracy of the network on the 2500 test images: 59 %
[4] loss: 1.149
12499
[5,  2000] loss: 1.023
[5,  4000] loss: 1.030
[5,  6000] loss: 1.015
[5,  8000] loss: 1.040
[5, 10000] loss: 1.043
[5, 12000] loss: 1.040
Accura

leakyrelu

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [8]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_leakyrelu")
visdom_accu = Visdom(env="testaccu_leakyrelu")
visdom_test = Visdom(env="testloss_leakyrelu")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_leakyrelu',
                    opts=dict(title='accu and loss',  legend=['train_loss_leakyrelu']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_leakyrelu']),
            name='test_accu_leakyrelu',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_leakyrelu']),
            name='test_loss_leakyrelu',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.201
[1,  4000] loss: 1.877
[1,  6000] loss: 1.681
[1,  8000] loss: 1.593
[1, 10000] loss: 1.536
[1, 12000] loss: 1.487
Accuracy of the network on the 2500 test images: 47 %
[1] loss: 1.443
12499
[2,  2000] loss: 1.421
[2,  4000] loss: 1.384
[2,  6000] loss: 1.363
[2,  8000] loss: 1.334
[2, 10000] loss: 1.314
[2, 12000] loss: 1.288
Accuracy of the network on the 2500 test images: 55 %
[2] loss: 1.256
12499
[3,  2000] loss: 1.218
[3,  4000] loss: 1.232
[3,  6000] loss: 1.234
[3,  8000] loss: 1.188
[3, 10000] loss: 1.195
[3, 12000] loss: 1.191
Accuracy of the network on the 2500 test images: 59 %
[3] loss: 1.165
12499
[4,  2000] loss: 1.094
[4,  4000] loss: 1.126
[4,  6000] loss: 1.122
[4,  8000] loss: 1.128
[4, 10000] loss: 1.126
[4, 12000] loss: 1.086
Accuracy of the network on the 2500 test images: 57 %
[4] loss: 1.177
12499
[5,  2000] loss: 1.042
[5,  4000] loss: 1.044
[5,  6000] loss: 1.032
[5,  8000] loss: 1.026
[5, 10000] loss: 1.051
[5, 12000] loss: 1.028
Accura

rrelu

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.rrelu(self.conv1(x)))
        x = self.pool(F.rrelu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.rrelu(self.fc1(x))
        x = F.rrelu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [16]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_rrelu")
visdom_accu = Visdom(env="testaccu_rrelu")
visdom_test = Visdom(env="testloss_rrelu")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_rrelu',
                    opts=dict(title='accu and loss',  legend=['train_loss_rrelu']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_rrelu']),
            name='test_accu_rrelu',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_rrelu']),
            name='test_loss_rrelu',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.181
[1,  4000] loss: 1.792
[1,  6000] loss: 1.583
[1,  8000] loss: 1.516
[1, 10000] loss: 1.444
[1, 12000] loss: 1.418
Accuracy of the network on the 2500 test images: 47 %
[1] loss: 1.433
12499
[2,  2000] loss: 1.329
[2,  4000] loss: 1.312
[2,  6000] loss: 1.289
[2,  8000] loss: 1.265
[2, 10000] loss: 1.222
[2, 12000] loss: 1.241
Accuracy of the network on the 2500 test images: 57 %
[2] loss: 1.210
12499
[3,  2000] loss: 1.161
[3,  4000] loss: 1.167
[3,  6000] loss: 1.156
[3,  8000] loss: 1.133
[3, 10000] loss: 1.139
[3, 12000] loss: 1.131
Accuracy of the network on the 2500 test images: 58 %
[3] loss: 1.171
12499
[4,  2000] loss: 1.067
[4,  4000] loss: 1.064
[4,  6000] loss: 1.074
[4,  8000] loss: 1.066
[4, 10000] loss: 1.058
[4, 12000] loss: 1.058
Accuracy of the network on the 2500 test images: 61 %
[4] loss: 1.105
12499
[5,  2000] loss: 0.991
[5,  4000] loss: 1.013
[5,  6000] loss: 0.980
[5,  8000] loss: 0.995
[5, 10000] loss: 1.036
[5, 12000] loss: 0.990
Accura

lr=0.001

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.rrelu(self.conv1(x)))
        x = self.pool(F.rrelu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.rrelu(self.fc1(x))
        x = F.rrelu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

Files already downloaded and verified
Files already downloaded and verified


In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  #随机梯度下降法

In [20]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.001")
visdom_accu = Visdom(env="testaccu_0.001")
visdom_test = Visdom(env="testloss_0.001")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.001',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.001']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.001']),
            name='test_accu_0.001',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.001']),
            name='test_loss_0.001',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.203
[1,  4000] loss: 1.861
[1,  6000] loss: 1.689
[1,  8000] loss: 1.580
[1, 10000] loss: 1.512
[1, 12000] loss: 1.452
Accuracy of the network on the 2500 test images: 47 %
[1] loss: 1.478
12499
[2,  2000] loss: 1.374
[2,  4000] loss: 1.344
[2,  6000] loss: 1.317
[2,  8000] loss: 1.285
[2, 10000] loss: 1.255
[2, 12000] loss: 1.250
Accuracy of the network on the 2500 test images: 57 %
[2] loss: 1.223
12499
[3,  2000] loss: 1.164
[3,  4000] loss: 1.165
[3,  6000] loss: 1.162
[3,  8000] loss: 1.151
[3, 10000] loss: 1.136
[3, 12000] loss: 1.121
Accuracy of the network on the 2500 test images: 60 %
[3] loss: 1.125
12499
[4,  2000] loss: 1.042
[4,  4000] loss: 1.050
[4,  6000] loss: 1.052
[4,  8000] loss: 1.053
[4, 10000] loss: 1.042
[4, 12000] loss: 1.055
Accuracy of the network on the 2500 test images: 63 %
[4] loss: 1.067
12499
[5,  2000] loss: 0.969
[5,  4000] loss: 0.967
[5,  6000] loss: 1.010
[5,  8000] loss: 0.985
[5, 10000] loss: 0.976
[5, 12000] loss: 0.967
Accura

0.0001

In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.rrelu(self.conv1(x)))
        x = self.pool(F.rrelu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.rrelu(self.fc1(x))
        x = F.rrelu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

Files already downloaded and verified
Files already downloaded and verified


In [22]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)  #随机梯度下降法

In [23]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.0001")
visdom_accu = Visdom(env="testaccu_0.0001")
visdom_test = Visdom(env="testloss_0.0001")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.0001',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.0001']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.0001']),
            name='test_accu_0.0001',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.0001']),
            name='test_loss_0.0001',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.304
[1,  4000] loss: 2.298
[1,  6000] loss: 2.289
[1,  8000] loss: 2.255
[1, 10000] loss: 2.186
[1, 12000] loss: 2.096
Accuracy of the network on the 2500 test images: 26 %
[1] loss: 2.063
12499
[2,  2000] loss: 2.052
[2,  4000] loss: 1.982
[2,  6000] loss: 1.932
[2,  8000] loss: 1.903
[2, 10000] loss: 1.867
[2, 12000] loss: 1.819
Accuracy of the network on the 2500 test images: 37 %
[2] loss: 1.777
12499
[3,  2000] loss: 1.785
[3,  4000] loss: 1.744
[3,  6000] loss: 1.696
[3,  8000] loss: 1.686
[3, 10000] loss: 1.662
[3, 12000] loss: 1.637
Accuracy of the network on the 2500 test images: 39 %
[3] loss: 1.679
12499
[4,  2000] loss: 1.592
[4,  4000] loss: 1.571
[4,  6000] loss: 1.560
[4,  8000] loss: 1.534
[4, 10000] loss: 1.511
[4, 12000] loss: 1.517
Accuracy of the network on the 2500 test images: 44 %
[4] loss: 1.523
12499
[5,  2000] loss: 1.475
[5,  4000] loss: 1.451
[5,  6000] loss: 1.453
[5,  8000] loss: 1.452
[5, 10000] loss: 1.435
[5, 12000] loss: 1.428
Accura

0.00001

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])#数据集加载时，默认的图片格式是 numpy，所以通过 transforms 转换成 Tensor。
                                                              #然后，再对输入图片进行标准化。
    #前面的（0.5，0.5，0.5） 是 R G B 三个通道上的均值， 后面(0.5, 0.5, 0.5)是三个通道的标准差，Normalize对每个通道执行以下操作：image =（图像-平均值）/ std

batch_size = 4

#batchsize: 批大小 num_works:num_works: 是否多进程读取数据 shuffle: 每个 epoch 是否乱序 drop_last: 当样本数不能被 batchsize 整除时，是否舍弃最后一批数据

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img*0.5 + 0.5     # unnormalize  还原被归一化的数据
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))    #在pytorch中张量tensor对图象的存储是(b,c,w,h)分别表示(图片数量，通道数，图片高，图片宽)。

                                                    #单独说tensor中的某张图片，也就是(管道数，宽，高)。而标准的rbg图象是(宽，高，管道数)。
    plt.show()
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)             #定义卷积层和池化层
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       #定义全连接层
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.rrelu(self.conv1(x)))
        x = self.pool(F.rrelu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.rrelu(self.fc1(x))
        x = F.rrelu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

Files already downloaded and verified
Files already downloaded and verified


In [25]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()         #用交叉熵作loss function
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)  #随机梯度下降法

In [26]:
from visdom import Visdom
visdom_show = Visdom(env="trainloss_0.00001")
visdom_accu = Visdom(env="testaccu_0.00001")
visdom_test = Visdom(env="testloss_0.00001")
for epoch in range(10):  # loop over the dataset multiple times
        #可视化

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data          

        # zero the parameter gradients
        optimizer.zero_grad()       #梯度从零开始

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
      

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if (i+1)==12000:
                visdom_show.line(
                    X=[epoch+1],
                    Y=[float(running_loss / 2000)],
                    win='accu and loss',
                    name='train_loss_0.00001',
                    opts=dict(title='accu and loss',  legend=['train_loss_0.00001']),
                    update='append')
            running_loss = 0.0
    correct = 0
    total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        running_loss2=0
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)        #放进模型里
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  #相同的累加
            loss = criterion(outputs, labels)
            running_loss2 += loss.item()
        print('Accuracy of the network on the 2500 test images: %d %%' % (100 * correct / total))
        visdom_accu.line(
            X=[epoch+1],
            Y=[(correct / total)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_accu_0.00001']),
            name='test_accu_0.00001',
            update='append')
        print('[%d] loss: %.3f' %
                  (epoch + 1,running_loss2 / 2500))
        visdom_test.line(
            X=[epoch+1],
            Y=[float(running_loss2 / 2500)],
            win='accu and loss',
            opts=dict(title='accu and loss',  legend=['test_loss_0.00001']),
            name='test_loss_0.00001',
            update='append')
        print(i)
print('Finished Training')

Setting up a new session...
Setting up a new session...
Setting up a new session...


[1,  2000] loss: 2.302
[1,  4000] loss: 2.303
[1,  6000] loss: 2.302
[1,  8000] loss: 2.301
[1, 10000] loss: 2.301
[1, 12000] loss: 2.300
Accuracy of the network on the 2500 test images: 10 %
[1] loss: 2.299
12499
[2,  2000] loss: 2.299
[2,  4000] loss: 2.298
[2,  6000] loss: 2.298
[2,  8000] loss: 2.296
[2, 10000] loss: 2.295
[2, 12000] loss: 2.293
Accuracy of the network on the 2500 test images: 16 %
[2] loss: 2.293
12499
[3,  2000] loss: 2.292
[3,  4000] loss: 2.291
[3,  6000] loss: 2.288
[3,  8000] loss: 2.287
[3, 10000] loss: 2.283
[3, 12000] loss: 2.281
Accuracy of the network on the 2500 test images: 20 %
[3] loss: 2.278
12499
[4,  2000] loss: 2.277
[4,  4000] loss: 2.272
[4,  6000] loss: 2.267
[4,  8000] loss: 2.262
[4, 10000] loss: 2.256
[4, 12000] loss: 2.248
Accuracy of the network on the 2500 test images: 20 %
[4] loss: 2.240
12499
[5,  2000] loss: 2.234
[5,  4000] loss: 2.226
[5,  6000] loss: 2.211
[5,  8000] loss: 2.197
[5, 10000] loss: 2.181
[5, 12000] loss: 2.170
Accura